In [1]:
%pip install conllu
%pip install stanza

import conllu
import pandas as pd
import stanza
from collections import defaultdict, Counter
from itertools import pairwise
import math

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/mnt/disk/patrik/nlp/.venv_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-train-la.conllu -O cs_pdtc-ud-train-la.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-test.conllu -O cs_pdtc-ud-test.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-dev.conllu -O cs_pdtc-ud-dev.conllu

!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-GUM/refs/heads/master/en_gum-ud-train.conllu -O en_gum-ud-train.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-GUM/refs/heads/master/en_gum-ud-test.conllu -O en_gum-ud-test.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-GUM/refs/heads/master/en_gum-ud-dev.conllu -O en_gum-ud-dev.conllu


--2026-02-09 21:25:31--  https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-train-la.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24175776 (23M) [text/plain]
Saving to: ‘cs_pdtc-ud-train-la.conllu’

cs_pdtc-ud-train-la 100%[===================>]  23,06M  46,8MB/s    in 0,5s    

2026-02-09 21:25:33 (46,8 MB/s) - ‘cs_pdtc-ud-train-la.conllu’ saved [24175776/24175776]

--2026-02-09 21:25:33--  https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.13

In [3]:
filenames_test = {
    "cs" : "cs_pdtc-ud-test.conllu",
    "en": "en_gum-ud-test.conllu"
}

filenames_dev = {
    "cs" : "cs_pdtc-ud-dev.conllu",
    "en": "en_gum-ud-dev.conllu"
}

filenames_train = {
    "cs" : "cs_pdtc-ud-train-la.conllu",
    "en": "en_gum-ud-train.conllu"
}

filenames = {
    "test": filenames_test,
    "dev": filenames_dev,
    "train": filenames_train
}

In [4]:
corpus = dict()

for name, d in filenames.items():
    corpus[name] = dict()
    for lang, filename in d.items():
        with open(filename, "r") as f:
            corpus[name][lang] = f.read()

In [5]:
nlp_en = stanza.Pipeline(lang='en', processors='tokenize,pos', tokenize_pretokenized=True)
nlp_cs = stanza.Pipeline(lang='cs', processors='tokenize,pos', tokenize_pretokenized=True)


2026-02-09 21:25:43 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-02-09 21:25:43 INFO: Downloaded file to /home/patrik/stanza_resources/resources.json
2026-02-09 21:25:45 INFO: Loading these models for language: en (English):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| pos       | combined_charlm |

2026-02-09 21:25:45 INFO: Using device: cpu
2026-02-09 21:25:45 INFO: Loading: tokenize
2026-02-09 21:25:45 INFO: Loading: pos
2026-02-09 21:25:51 INFO: Done loading processors!
2026-02-09 21:25:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-02-09 21:25:51 INFO: Downloaded file to /home/patrik/stanza_resources/resources.json
2026-02-09

In [6]:
# MANUAL FUNCTIONS - FOR DEBUGGING AND STUFF - NOT USED CURRENTLY

def line_number_check(data, text):
    i=0
    for line in data.split("\n"):
        splitted = line.split("\t")
        if len(splitted) == 0 or splitted[0].startswith("#") or "-" in splitted[0] or "." in splitted[0] or splitted[0] == "\n" or splitted[0] == "":
            continue
        i += 1
    print(f"{text} size : {i}")


def word_tag_count(data, text):
    word_tag_pairs = set()
    i = 0
    for line in data.split("\n"):
        splitted = line.split("\t")
        if len(splitted) == 0 or splitted[0].startswith("#") or "-" in splitted[0] or "." in splitted[0] or splitted[0] == "\n" or splitted[0] == "":
            continue
        word_tag_pairs.add((splitted[1], splitted[4]))
        i+=1

    print(f"{text} size : {i}")

def word_tag_count(data, text):
    word_tag_pairs = set()
    i = 0
    for line in data.split("\n"):
        splitted = line.split("\t")
        if len(splitted) == 0 or splitted[0].startswith("#") or "-" in splitted[0] or "." in splitted[0] or splitted[0] == "\n" or splitted[0] == "":
            continue
        word_tag_pairs.add((splitted[1], splitted[4]))
        i+=1

    print(f"{text} size : {i}")
    # print(f"{text} word-tag pairs : {len(word_tag_pairs)}")

# line_number_check(corpus["train"]["en"], "English train")
# line_number_check(corpus["dev"]["cs"], "Czech dev")
# line_number_check(corpus["test"]["cs"], "Czech test")

# word_tag_count(corpus["test"]["en"], "English test")
# word_tag_count("\n\n".join(corpus["test"]["cs"].split("\n\n")[:1500]), "Czech test")

In [7]:
def tagize(filename, text, max_sentences=999_999_999):
    tokens = []
    tags = []
    sentence_count=0
    tag_count = 0
    with open (filename, "r") as f:
        for sentence in conllu.parse_incr(f):
            sentence_tokens = []
            sentence_tags = []
            if sentence_count >= max_sentences:
                break
            for token in sentence:
                if isinstance(token["id"], (list, tuple)) and ("." in token["id"] or "-" in token["id"]):
                    continue
                # print(token["form"], token["upos"])
                tag_count += 1
                sentence_tokens.append(token["form"])
                sentence_tags.append(token["upos"])
            
            sentence_count += 1
            tokens.append(sentence_tokens)
            tags.append(sentence_tags)
    print(text, "sentence count:", sentence_count, "word/tag pairs (lines):", tag_count)
    return tokens, tags


print("BEFORE TRUNCATION TO 1_500 SENTENCES")
pairs = {type: {lang: tagize(filename, f"{type}-{lang}") for lang, filename in d.items()} for type, d in filenames.items()}

BEFORE TRUNCATION TO 1_500 SENTENCES
test-cs sentence count: 20187 word/tag pairs (lines): 305808
test-en sentence count: 1464 word/tag pairs (lines): 28397
dev-cs sentence count: 22666 word/tag pairs (lines): 384431
dev-en sentence count: 1575 word/tag pairs (lines): 28119
train-cs sentence count: 12519 word/tag pairs (lines): 218409
train-en sentence count: 10224 word/tag pairs (lines): 177410


In [8]:
print("TRUNCATING TEST SETS")
pairs["test"] = {lang: tagize(filename, f"test-{lang}", max_sentences=1500) for lang, filename in filenames["test"].items()}
tokens = {type: {lang: pair[0] for lang, pair in d.items()} for type, d in pairs.items()}
tags = {type: {lang: pair[1] for lang, pair in d.items()} for type, d in pairs.items()}

TRUNCATING TEST SETS
test-cs sentence count: 1500 word/tag pairs (lines): 22844
test-en sentence count: 1464 word/tag pairs (lines): 28397


In [9]:
doc_en = nlp_en(tokens["test"]["en"])
doc_cs = nlp_cs(tokens["test"]["cs"][:1500])

In [10]:
def stanza_eval(doc, tokens_sents, gold_sents):
    """tokens_sents: List[List[str]], gold_sents: List[List[str]] (UPOS)"""
    pred_sents = [[w.upos for w in s.words] for s in doc.sentences]

    # accuracy
    correct = total = 0
    for g, p in zip(gold_sents, pred_sents):
        if len(g) != len(p):
            raise ValueError(f"Length mismatch in a sentence: gold={len(g)} pred={len(p)} "
                             "(check MWT/ellipsis filtering + tokenize_pretokenized=True)")
        for gt, pt in zip(g, p):
            total += 1
            correct += (gt == pt)
    acc = 100.0 * correct / total

    # confusion matrix (rows=gold, cols=pred)
    gflat = [t for sent in gold_sents for t in sent]
    pflat = [t for sent in pred_sents for t in sent]
    cm = pd.crosstab(pd.Series(gflat, name="gold"), pd.Series(pflat, name="pred"))
    return acc, cm

en_acc, en_cm = stanza_eval(doc_en, tokens["test"]["en"], tags["test"]["en"])
print(f"Stanza EN test accuracy: {en_acc:.4f}%")
print("Stanza EN confusion matrix below")
display(en_cm)

cs_acc, cs_cm = stanza_eval(doc_cs, tokens["test"]["cs"][:1500], tags["test"]["cs"][:1500])
print(f"Stanza CS test accuracy: {cs_acc:.4f}%")
print("Stanza CS confusion matrix below")
display(cs_cm)


Stanza EN test accuracy: 97.8308%
Stanza EN confusion matrix below


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,1755,2,11,0,0,0,0,34,0,0,0,13,0,0,0,18,0
ADP,0,2833,12,0,2,0,0,0,0,1,0,2,0,15,0,0,0
ADV,37,36,1226,0,0,14,6,8,0,0,3,1,0,4,0,1,0
AUX,0,0,0,1520,0,0,0,0,0,0,0,0,0,0,0,13,0
CCONJ,0,0,0,0,987,0,0,0,0,0,0,0,0,1,0,0,0
DET,0,0,1,0,3,2515,1,0,0,0,1,0,0,0,0,0,0
INTJ,1,4,12,0,1,0,155,2,0,0,0,1,0,3,0,2,3
NOUN,25,0,0,1,0,0,2,4806,3,0,1,59,0,0,0,8,1
NUM,0,1,0,0,0,0,0,0,478,0,0,0,0,0,0,0,0


Stanza CS test accuracy: 97.2903%
Stanza CS confusion matrix below


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,2572,0,1,0,0,0,0,16,1,0,0,11,0,0,0,0,1
ADP,0,2109,0,0,0,0,0,3,0,0,1,0,0,0,0,1,0
ADV,1,2,878,0,16,1,0,10,0,5,3,0,0,0,0,1,0
AUX,0,0,0,774,0,0,0,0,0,0,0,0,0,0,0,2,0
CCONJ,0,0,12,0,731,0,0,0,0,2,0,0,0,11,0,0,0
DET,1,0,1,0,0,916,0,0,0,0,0,0,0,0,0,1,0
INTJ,0,0,0,0,0,0,14,2,0,3,0,1,0,0,0,0,0
NOUN,12,3,7,0,4,0,4,5525,0,0,0,32,0,0,6,3,4
NUM,0,0,0,0,0,0,0,0,616,0,0,0,0,0,0,0,0


In [11]:
print("TRUNKATING CS DEV SET")
tokens["dev"]["cs"], tags["dev"]["cs"] = tagize(filenames["dev"]["cs"], "dev-cs", max_sentences=1500)

TRUNKATING CS DEV SET


dev-cs sentence count: 1500 word/tag pairs (lines): 21752


In [12]:
# Check sizes
print("Sizes check")
for type, d in tokens.items():
    for lang, sents in d.items():
        print(f"{type}-{lang} tokenized sentences: {len(sents)} tokens : {sum(len(s) for s in sents)}")

Sizes check
test-cs tokenized sentences: 1500 tokens : 22844
test-en tokenized sentences: 1464 tokens : 28397
dev-cs tokenized sentences: 1500 tokens : 21752
dev-en tokenized sentences: 1575 tokens : 28119
train-cs tokenized sentences: 12519 tokens : 218409
train-en tokenized sentences: 10224 tokens : 177410


# Transition model smoothing

In [13]:
BOS = "<s>"
EOS = "</s>"

# UPDATED BIGRAMS SO THERE IS ALSO start_symbol followed by start_symbol (and also end followed by end)
get_unigrams = lambda tokens : Counter(tokens + [EOS])
get_bigrams = lambda tokens : Counter(zip(2 * [BOS] + tokens + [EOS], [BOS] + tokens + 2 * [EOS]))
get_trigrams = lambda tokens : Counter(zip(2 * [BOS] + tokens, [BOS] + tokens + [EOS], tokens + 2 * [EOS]))

def get_all_ngrams(tokens_or_tags, n=1):
    ngrams = {
        1: get_unigrams,
        2: get_bigrams,
        3: get_trigrams
    }[n]
    all = Counter()    
    for sentence in tokens_or_tags:
        all.update(ngrams(sentence))
    return all


In [14]:
# COPIED FROM THE FIRST ASSIGNMENT
# NOT IDEA IF THIS IS CORRECT SINCE I REVIECED NO FEEDBACK :)
# BUT I GOT 92 POINTS SO HOPEFULLY THIS IS OK

def bigram_probab(bigrams, unigrams, wi, wi_1):
    c2 = bigrams[(wi_1, wi)]#  if (wi_1, wi) in bigrams else 0
    c1 = unigrams[wi_1]# if wi_1 in unigrams else 0
    if c1 == 0:
        return 1 / len(unigrams)
    return c2 / c1

def trigram_probab(trigrams, bigrams, unigrams, wi, wi_1, wi_2):
    c3 = trigrams[(wi_2, wi_1, wi)]# if (wi_2, wi_1, wi) in trigrams else 0
    c2 = bigrams[(wi_2, wi_1)]# if (wi_2, wi_1) in bigrams else 0
    if c2 == 0:
        return 1 / len(unigrams)
    return c3 / c2

def smoothed_probab_trigram(trigrams, bigrams, unigrams, wi, wi_1, wi_2, lambdas, N_uni = None):
    N_uni = sum(unigrams.values()) if N_uni == None else N_uni
    lambda0, lambda1, lambda2, lambda3 = lambdas
    p0 = 1 / len(unigrams)
    p1 = unigrams[wi] / N_uni# if wi in unigrams else 1 / sum(unigrams.values())
    p2 = bigram_probab(bigrams, unigrams, wi, wi_1)
    p3 = trigram_probab(trigrams, bigrams, unigrams, wi, wi_1, wi_2)
    return lambda0 * p0 + lambda1 * p1 + lambda2 * p2 + lambda3 * p3


def EM_smoothing(train, heldout, max_iter=100, e = 1e-4):
    lambdas = [0.25, 0.25, 0.25, 0.25]

    # --- ONE MINOR CHANGE FROM THE FIRST ASSIGNMENT ---
    # trigrams_heldout = get_trigrams(heldout)
    # bigrams_heldout = get_bigrams(heldout)
    # unigrams_heldout = get_unigrams(heldout)

    # trigrams_train = get_trigrams(train)
    # bigrams_train = get_bigrams(train)
    # unigrams_train = get_unigrams(train)

    trigrams_heldout = get_all_ngrams(heldout, n=3)
    bigrams_heldout = get_all_ngrams(heldout, n=2)
    unigrams_heldout = get_all_ngrams(heldout, n=1)

    trigrams_train = get_all_ngrams(train, n=3)
    bigrams_train = get_all_ngrams(train, n=2)
    unigrams_train = get_all_ngrams(train, n=1)
    # --- --- ----- ------ ---- --- ----- ---------- ---

    N_uni = sum(unigrams_train.values()) # saves a lot of time lol

    iter = 0
    while iter < max_iter:
        counts = [0.0, 0.0, 0.0, 0.0]

        for (wi_2, wi_1, wi), c in trigrams_heldout.items():
            p0 = 1 / len(unigrams_train)
            p1 = unigrams_train[wi] / N_uni# if wi in unigrams_train else 1 / N_uni
            p2 = bigram_probab(bigrams_train, unigrams_train, wi, wi_1)
            p3 = trigram_probab(trigrams_train, bigrams_train, unigrams_train, wi, wi_1, wi_2)

            p_total = lambdas[0] * p0 + lambdas[1] * p1 + lambdas[2] * p2 + lambdas[3] * p3

            counts[0] += c * (lambdas[0] * p0) / p_total
            counts[1] += c * (lambdas[1] * p1) / p_total
            counts[2] += c * (lambdas[2] * p2) / p_total
            counts[3] += c * (lambdas[3] * p3) / p_total

        new_lambdas = [count / sum(counts) for count in counts]

        # print("Iteration ", iter, " lambdas: ", new_lambdas)

        if all(abs(new_lambdas[i] - lambdas[i]) < e for i in range(4)):
            break

        lambdas = new_lambdas
        iter += 1
    return lambdas

In [15]:
ngrams_train_tag = {type: {i: {lang: get_all_ngrams(tags["train"][lang], n=i) for lang in tags["train"]} for i in range(1, 4)} for type in tags["train"]}

In [16]:
em_lambdas = {lang: EM_smoothing(tags["train"][lang], tags["dev"][lang]) for lang in tags["train"]}

In [17]:
em_lambdas

{'cs': [0.005635399093115278,
  0.005527962553778706,
  0.10105815981569118,
  0.8877784785374149],
 'en': [0.00012675633014343153,
  0.0033791000786512265,
  0.08104221881633325,
  0.915451924774872]}

# Emission model smoothing

In [18]:
from collections import Counter
import math

UNK = "<UNK>"

def train_lexical_model(tokens_train, tags_train, tokens_dev, tags_dev,
                        unk_threshold=1, add_alpha=0.0, eps=1e-6, max_iter=80):
    """
    Learns interpolated emission model with UNK:

      P(w|t) = λ0 * 1/|V|  +  λ1 * P(w)  +  λ2 * P_MLE(w|t)

    - UNK: words with train freq <= unk_threshold become <UNK> in training,
           unseen words at decode time also become <UNK>.
    - Lambdas (λ0,λ1,λ2) learned on DEV with EM-like expected-counts updates.

    Returns dict with:
      - logp_emit(tag, word): log P(word|tag)
      - lambdas: [λ0, λ1, λ2]
      - vocab: normalized vocab (includes <UNK>)
    """

    # --- TRAIN vocab + UNK mapping ---
    wf = Counter(w for sent in tokens_train for w in sent)

    def norm_train(w):
        return w if wf[w] > unk_threshold else UNK

    emit = Counter()         # (t, w)
    tag_count = Counter()    # t
    word_count = Counter()   # w
    vocab = {UNK}
    total = 0

    for words, tags in zip(tokens_train, tags_train):
        for w, t in zip(words, tags):
            w2 = norm_train(w)
            vocab.add(w2)
            emit[(t, w2)] += 1
            tag_count[t] += 1
            word_count[w2] += 1
            total += 1

    V = len(vocab)

    def p0(w):  # uniform over vocab
        return 1.0 / V

    def p1(w):  # unigram P(w)
        denom = total + add_alpha * V
        return (word_count[w] + add_alpha) / denom if denom else 0.0

    def p2(t, w):  # MLE P(w|t)
        denom = tag_count[t] + add_alpha * V
        return (emit[(t, w)] + add_alpha) / denom if denom else 0.0

    def norm_dev(w):
        # unseen -> UNK
        return w if w in vocab else UNK

    # --- learn lambdas on DEV ---
    lambdas = [1/3, 1/3, 1/3]
    for _ in range(max_iter):
        exp = [0.0, 0.0, 0.0]

        for words, tags in zip(tokens_dev, tags_dev):
            for w_raw, t in zip(words, tags):
                w = norm_dev(w_raw)
                probs = [p0(w), p1(w), p2(t, w)]
                mix = [lambdas[i] * probs[i] for i in range(3)]
                Z = sum(mix)
                if Z <= 0:
                    continue
                exp[0] += mix[0] / Z
                exp[1] += mix[1] / Z
                exp[2] += mix[2] / Z

        s = sum(exp)
        new_lambdas = [e / s for e in exp] if s else lambdas
        if max(abs(new_lambdas[i] - lambdas[i]) for i in range(3)) < eps:
            lambdas = new_lambdas
            break
        lambdas = new_lambdas

    def logp_emit(t, w_raw):
        w = w_raw if w_raw in vocab else UNK
        p = lambdas[0] * p0(w) + lambdas[1] * p1(w) + lambdas[2] * p2(t, w)
        return -math.inf if p <= 0.0 else math.log(p)

    return {"vocab": vocab, "lambdas": lambdas, "logp_emit": logp_emit,
            "emit": emit, "tag_count": tag_count, "word_count": word_count}


In [19]:
lex = {}
for lang in ["en", "cs"]:
    lex[lang] = train_lexical_model(
        tokens["train"][lang], tags["train"][lang],
        tokens["dev"][lang],   tags["dev"][lang],
        unk_threshold=1, add_alpha=0.0
    )


# HMM/Viterbi

In [24]:
import numpy as np

def build_tag_index(tagset, BOS="<s>", EOS="</s>"):
    tags = [t for t in tagset if t not in (BOS, EOS)]
    idx = {t:i for i,t in enumerate(tags)}
    return tags, idx

def precompute_trans_tensor(tags, logp_trans, BOS="<s>", EOS="</s>"):
    """
    Returns:
      A: (T+1, T+1, T) where indices 0..T-1 are real tags, index T is BOS2
      end1: (T, T) for log P(EOS | u,v)
      end2: (T,) for log P(EOS | v, EOS) or log P(EOS | v, EOS) depending on your transition definition
    """
    T = len(tags)
    BOSi = T  # extra index for BOS
    A = np.full((T+1, T+1, T), -np.inf, dtype=np.float64)

    # transitions to real tags
    for ui in range(T+1):
        u = BOS if ui == BOSi else tags[ui]
        for vi in range(T+1):
            v = BOS if vi == BOSi else tags[vi]
            for ci in range(T):
                c = tags[ci]
                A[ui, vi, ci] = logp_trans(u, v, c)

    # termination pieces
    end1 = np.full((T, T), -np.inf, dtype=np.float64)  # log P(EOS | u,v)
    end2 = np.full((T,), -np.inf, dtype=np.float64)    # log P(EOS | v, EOS) or log P(EOS | v, EOS)
    for ui in range(T):
        u = tags[ui]
        for vi in range(T):
            v = tags[vi]
            end1[ui, vi] = logp_trans(u, v, EOS)
    for vi in range(T):
        v = tags[vi]
        end2[vi] = logp_trans(v, EOS, EOS)

    return A, end1, end2, BOSi

def viterbi_trigram_fast(words, tagset, logp_trans, logp_emit, BOS="<s>", EOS="</s>"):
    """
    Trigram Viterbi in numpy. Same result as your Python version, much faster.
    """
    if not words:
        return []

    tags, idx = build_tag_index(tagset, BOS=BOS, EOS=EOS)
    T = len(tags)

    A, end1, end2, BOSi = precompute_trans_tensor(tags, logp_trans, BOS=BOS, EOS=EOS)

    # dp[pp, p] corresponds to best score ending with (t_{i-1}=pp, t_i=p) at current position i
    dp = np.full((T+1, T+1), -np.inf, dtype=np.float64)
    dp[BOSi, BOSi] = 0.0

    # backpointers: for each position i store argmax over pp for each (p,c)
    bps = []

    for w in words:
        emit = np.array([logp_emit(t, w) for t in tags], dtype=np.float64)  # (T,)
        # scores[pp, p, c] = dp[pp,p] + A[pp,p,c] + emit[c]
        scores = dp[:, :, None] + A + emit[None, None, :]
        # maximize over pp dimension -> new_dp[p,c]
        new_dp = np.max(scores, axis=0)  # (T+1, T) but only p in 0..T and c in 0..T-1
        arg_pp = np.argmax(scores, axis=0)  # same shape

        # next dp should be over (p,c) pairs => dp_next[p,c]
        dp = np.full((T+1, T+1), -np.inf, dtype=np.float64)
        dp[:, :T] = new_dp  # dp[p, c] lives in dp[p, c]
        bps.append(arg_pp)  # store argmax pp for each (p,c)

    # termination: choose best (u,v) among real tags (0..T-1)
    # dp[u,v] currently stored in dp[u,v] where both are in 0..T-1
    best_score = -np.inf
    best_u = best_v = 0
    for u in range(T):
        for v in range(T):
            s = dp[u, v] + end1[u, v] + end2[v]
            if s > best_score:
                best_score = s
                best_u, best_v = u, v

    # backtrack
    n = len(words)
    out = [None] * n
    if n == 1:
        out[0] = tags[best_v]
        return out

    out[-2] = tags[best_u]
    out[-1] = tags[best_v]

    u, v = best_u, best_v
    for i in range(n-1, 1, -1):
        # bps[i] corresponds to word i+1 (1-indexed); we need arg_pp for (p=u?, c=v?)
        arg_pp = bps[i][u, v]  # argmax pp for this (u,v)
        out[i-2] = tags[arg_pp] if arg_pp < T else BOS
        v, u = u, arg_pp  # shift window backwards

    return out


In [ ]:
def take_first_n_pairs(tokens_sents, tags_sents, n_pairs):
    out_tok, out_tag = [], []
    seen = 0
    for words, tags in zip(tokens_sents, tags_sents):
        if seen >= n_pairs:
            break
        need = n_pairs - seen
        if len(words) <= need:
            out_tok.append(words)
            out_tag.append(tags)
            seen += len(words)
        else:
            out_tok.append(words[:need])
            out_tag.append(tags[:need])
            seen += need
            break
    return out_tok, out_tag

def build_logp_trans_from_train_tags(lang, tags_train_sents):
    uni = get_all_ngrams(tags_train_sents, n=1)
    bi  = get_all_ngrams(tags_train_sents, n=2)
    tri = get_all_ngrams(tags_train_sents, n=3)
    N_uni = sum(uni.values())
    lambdas = em_lambdas[lang]

    def logp_trans(u, v, t):
        p = smoothed_probab_trigram(tri, bi, uni, wi=t, wi_1=v, wi_2=u, lambdas=lambdas, N_uni=N_uni)
        return -math.inf if p <= 0.0 else math.log(p)

    tagset = [t for t in uni.keys() if t not in (BOS, EOS)]
    return logp_trans, tagset

def eval_tagging(gold_sents, pred_sents):
    gold_flat, pred_flat = [], []
    for i, (g, p) in enumerate(zip(gold_sents, pred_sents)):
        if len(g) != len(p):
            raise ValueError(f"Sentence {i} length mismatch: gold={len(g)} pred={len(p)}")
        gold_flat.extend(g)
        pred_flat.extend(p)
    acc = 100.0 * sum(gt == pt for gt, pt in zip(gold_flat, pred_flat)) / len(gold_flat)
    cm = pd.crosstab(pd.Series(gold_flat, name="gold"), pd.Series(pred_flat, name="pred"))
    return acc, cm



In [25]:
def build_supervised_hmm(lang, tokens_train, tags_train, tokens_dev, tags_dev,
                         unk_threshold=1, add_alpha=0.0):
    # transitions (use your existing interpolation lambdas em_lambdas[lang])
    logp_trans, tagset = build_logp_trans_from_train_tags(lang, tags_train)

    # lexical model (interpolated, lambdas learned on dev)
    lex_model = train_lexical_model(tokens_train, tags_train, tokens_dev, tags_dev,
                                    unk_threshold=unk_threshold, add_alpha=add_alpha)
    return logp_trans, tagset, lex_model

def decode_and_eval(lang, tokens_sents, gold_tags_sents, logp_trans, tagset, lex_model):
    pred = [
        viterbi_trigram_fast(sent, tagset, logp_trans, lex_model["logp_emit"], BOS=BOS, EOS=EOS)
        for sent in tokens_sents
    ]
    acc, cm = eval_tagging(gold_tags_sents, pred)
    return acc, cm, pred


In [26]:
# English full
logp_trans_en, tagset_en, lex_en = build_supervised_hmm(
    "en",
    tokens["train"]["en"], tags["train"]["en"],
    tokens["dev"]["en"],   tags["dev"]["en"],
    unk_threshold=1, add_alpha=0.0
)
en_acc, en_cm, _ = decode_and_eval("en",
    tokens["test"]["en"], tags["test"]["en"],
    logp_trans_en, tagset_en, lex_en
)
print("FULL supervised HMM EN test accuracy:", f"{en_acc:.4f}")
display(en_cm)
print("EN lexical lambdas:", lex_en["lambdas"])


# Czech full (remember: test usually first 1500 sentences)
logp_trans_cs, tagset_cs, lex_cs = build_supervised_hmm(
    "cs",
    tokens["train"]["cs"], tags["train"]["cs"],
    tokens["dev"]["cs"],   tags["dev"]["cs"],
    unk_threshold=1, add_alpha=0.0
)
cs_acc, cs_cm, _ = decode_and_eval("cs",
    tokens["test"]["cs"][:1500], tags["test"]["cs"][:1500],
    logp_trans_cs, tagset_cs, lex_cs
)
print("FULL supervised HMM CS test accuracy:", f"{cs_acc:.4f}")
display(cs_cm)
print("CS lexical lambdas:", lex_cs["lambdas"])


FULL supervised HMM EN test accuracy: 90.8969


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,1557,1,36,0,0,0,2,97,1,0,0,87,1,0,0,46,5
ADP,0,2756,29,0,2,0,2,1,0,33,0,4,0,33,0,4,1
ADV,49,49,1102,0,0,13,33,31,0,0,18,17,0,6,0,17,1
AUX,0,0,0,1492,0,0,0,6,0,1,0,0,0,0,0,34,0
CCONJ,0,0,5,0,970,12,0,0,0,0,0,0,0,1,0,0,0
DET,2,1,23,0,5,2443,4,3,0,0,37,0,0,2,0,1,0
INTJ,1,3,9,0,0,0,152,1,3,0,0,3,1,4,0,6,1
NOUN,124,1,16,1,0,1,5,4305,5,0,2,300,1,0,0,129,16
NUM,11,0,1,0,0,0,0,28,365,1,6,62,0,0,0,2,3


EN lexical lambdas: [0.004517766102101085, 0.00020438802734153852, 0.9952778458705575]
FULL supervised HMM CS test accuracy: 88.2507


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,2323,1,1,0,1,0,0,120,1,0,0,39,0,0,0,112,4
ADP,1,2078,0,0,0,0,0,2,0,0,30,0,0,0,0,2,1
ADV,29,0,761,0,15,1,0,35,0,2,8,9,0,26,0,31,0
AUX,8,0,0,749,0,0,0,6,0,0,2,0,0,0,0,11,0
CCONJ,1,0,17,0,728,0,0,1,1,1,0,0,0,6,0,1,0
DET,34,0,1,0,0,871,0,5,0,0,0,1,1,0,0,6,0
INTJ,0,0,0,0,0,0,3,0,0,0,0,12,0,0,0,3,2
NOUN,391,10,6,0,11,0,3,4787,16,0,0,200,0,0,0,152,24
NUM,30,0,0,0,0,0,0,30,519,0,0,33,0,0,0,4,0


CS lexical lambdas: [3.5398104972372755e-07, 0.010711851767798203, 0.9892877942511521]


In [27]:
# ---- 10k SUPERVISED INIT (EN) ----
tok10k_en, tag10k_en = take_first_n_pairs(tokens["train"]["en"], tags["train"]["en"], 10_000)
print("EN train token pairs used:", sum(len(s) for s in tok10k_en))

# transitions built from the 10k subset (but still using em_lambdas["en"])
logp_trans_10k_en, tagset_10k_en = build_logp_trans_from_train_tags("en", tag10k_en)

# lexical model built from the 10k subset, lambdas learned on full dev
lex_10k_en = train_lexical_model(tok10k_en, tag10k_en,
                                 tokens["dev"]["en"], tags["dev"]["en"],
                                 unk_threshold=1, add_alpha=0.0)
print("EN 10k lexical lambdas:", lex_10k_en["lambdas"])

# decode + evaluate on EN test
en10k_acc, en10k_cm, _ = decode_and_eval("en",
    tokens["test"]["en"], tags["test"]["en"],
    logp_trans_10k_en, tagset_10k_en, lex_10k_en
)
print("10k-init supervised HMM EN test accuracy:", f"{en10k_acc:.4f}")
display(en10k_cm)


EN train token pairs used: 10000
EN 10k lexical lambdas: [4.3995548351829634e-07, 0.08179960365358227, 0.9181999563909342]
10k-init supervised HMM EN test accuracy: 70.9582


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,1093,3,47,1,1,2,0,233,5,0,0,221,4,0,2,212,9
ADP,22,2527,23,3,2,7,0,46,0,65,1,45,11,45,0,65,3
ADV,91,57,636,2,2,19,0,187,10,0,34,134,14,17,1,128,4
AUX,20,3,9,1158,2,3,0,48,0,52,4,106,14,3,0,110,1
CCONJ,12,3,34,1,886,6,0,5,0,1,6,19,2,0,0,13,0
DET,22,7,4,5,11,2345,0,22,0,0,42,31,8,10,0,11,3
INTJ,20,7,75,0,0,1,1,19,3,0,2,38,1,2,0,11,4
NOUN,386,1,48,4,0,2,0,3494,29,0,9,614,12,1,2,287,17
NUM,43,0,3,1,0,5,0,72,226,1,21,76,8,0,0,20,3


In [28]:
# ---- 10k SUPERVISED INIT (EN) ----
tok10k_cs, tag10k_cs = take_first_n_pairs(tokens["train"]["cs"], tags["train"]["cs"], 10_000)
print("CS train token pairs used:", sum(len(s) for s in tok10k_cs))

# transitions built from the 10k subset (but still using em_lambdas["cs"])
logp_trans_10k_cs, tagset_10k_cs = build_logp_trans_from_train_tags("cs", tag10k_cs)

# lexical model built from the 10k subset, lambdas learned on full dev
lex_10k_cs = train_lexical_model(tok10k_cs, tag10k_cs,
                                 tokens["dev"]["cs"], tags["dev"]["cs"],
                                 unk_threshold=1, add_alpha=0.0)
print("CS 10k lexical lambdas:", lex_10k_cs["lambdas"])

# decode + evaluate on CS test
cs10k_acc, cs10k_cm, _ = decode_and_eval("cs",
    tokens["test"]["cs"], tags["test"]["cs"],
    logp_trans_10k_cs, tagset_10k_cs, lex_10k_cs
)
print("10k-init supervised HMM CS test accuracy:", f"{cs10k_acc:.4f}")
display(cs10k_cm)

CS train token pairs used: 10000
CS 10k lexical lambdas: [4.044699664731544e-10, 0.0239036047924698, 0.9760963948030603]
10k-init supervised HMM CS test accuracy: 68.3549


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,1692,1,22,0,1,4,0,532,17,0,1,62,2,0,1,191,76
ADP,35,1992,2,0,0,0,0,24,0,0,36,2,0,0,1,18,4
ADV,158,0,379,0,8,9,0,155,2,6,1,14,2,23,3,146,11
AUX,21,1,2,694,0,1,0,27,0,0,3,0,2,4,0,20,1
CCONJ,13,0,13,0,686,3,0,10,0,3,0,3,0,7,0,15,3
DET,187,0,13,0,0,597,0,59,1,7,0,2,2,2,1,39,9
INTJ,0,0,0,0,0,0,1,0,0,0,0,15,0,0,0,4,0
NOUN,1093,18,30,3,12,7,7,3580,34,1,4,230,1,0,5,402,173
NUM,148,0,2,0,1,0,0,100,268,0,0,41,5,0,2,27,22
